In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import supabase_client
from src.update_pipeline import update_training_week, _update_training_week
from src.types.update_pipeline import ExeType
from src import auth_manager
from src.utils import datetime_now_est, get_last_sunday

import os
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

# New Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
assert user.preferences.race_date is not None
assert user.preferences.race_distance is not None

_update_training_week(user, ExeType.NEW_WEEK, dt=get_last_sunday())

# Mid Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
response = update_training_week(user, ExeType.MID_WEEK, dt=datetime_now_est())